In [2]:
import os

os.sys.path.insert(0, '../script')

from just_join import *
from evaluation import *

# Model

In [2]:
model = JustJoinGenerator()

In [23]:
evaluate_model(model, 'just-join')

{'bleu': 11.62, 'meteor': 0.2631907602030226, 'ter': 0.7995305073983557}

## Individual samples

In [25]:
sample = test.sample(ntriples=[5], seed=200)
sample

Triple info: Category=City eid=Id806 idx=test_City_5_Id806

	Modified Triples:

Austin,_Texas | isPartOf | Texas
Texas | language | English_language
Austin,_Texas | isPartOf | Williamson_County,_Texas
Williamson_County,_Texas | largestCity | Round_Rock,_Texas
Williamson_County,_Texas | countySeat | Georgetown,_Texas


	Lexicalizations:

Austin is part of Williamson County, in Texas, where English is a language spoken. Round Rock is the largest city in Williamson County and Georgetown is the county seat.


Austin is located in Texas which is where English is spoken. Austin is part of Williamson County, Texas and the largest city is Round Rock while Georgetown is the county seat.


In [27]:
model.predict([sample.data])[0]

'Austin,_Texas isPartOf Texas,Texas language English_language,Austin,_Texas isPartOf Williamson_County,_Texas,Williamson_County,_Texas largestCity Round_Rock,_Texas,Williamson_County,_Texas countySeat Georgetown,_Texas'

# Variantions

In [3]:
model = JustJoinGenerator(preprocess_subject=parenthesis_underline_camelcase,
                          preprocess_predicate=parenthesis_underline_camelcase,
                          preprocess_object=parenthesis_underline_camelcase,
                          sen_sep=', ')

evaluate_model(model, 'just-join-parenthesis_underline_camelcase')

{'bleu': 26.13, 'meteor': 0.34194546816488575, 'ter': 0.680779665582728}